In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [2]:
# Import file
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# Use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type_counts[app_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4500        1
C1570        1
C6100        1
C2170        1
C2500        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_1 = application_df['CLASSIFICATION'].value_counts()

print(classification_counts_1[classification_counts_1 > 1])

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C4100        6
C2400        6
C1720        6
C1257        5
C1600        5
C2710        3
C0           3
C1260        3
C3200        2
C1234        2
C1256        2
C1246        2
C1267        2
Name: CLASSIFICATION, dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# Use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
data=pd.get_dummies(application_df)

In [11]:
# Split our preprocessed data into our features and target arrays
y = data["IS_SUCCESSFUL"].values
X = data.drop(["IS_SUCCESSFUL"], axis = 1).values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Attempt #1 to Optimize

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
start = len(X_train[0])
h1 = 100
h2 = 50
h3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=start, units=h1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=h2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=h3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               4500      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 26        
Total params: 10,851
Trainable params: 10,851
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 1ms/step - loss: 48126.1250 - accuracy: 0.4857
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 8805.7891 - accuracy: 0.4882
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 16883.8555 - accuracy: 0.5035
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 14027.6768 - accuracy: 0.4968
Epoch 5/50
804/804 [==============================] - 1s 1ms/step - loss: 7948.0552 - accuracy: 0.4976
Epoch 6/50
804/804 [==============================] - 1s 1ms/step - loss: 2099.2246 - accuracy: 0.4930
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 3816.1643 - accuracy: 0.4976
Epoch 8/50
804/804 [==============================] - 1s 688us/step - loss: 5808.5786 - accuracy: 0.4935
Epoch 9/50
804/804 [==============================] - 1s 705us/step - loss: 5904.0298 - accuracy: 0.4822
Epoch 10/50
804/804 [==============================] - 1s 671us/st

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7287 - accuracy: 0.5248
Loss: 0.7287497520446777, Accuracy: 0.524781346321106


# Attempt #2 to Optimize BEST MODEL 70%

In [118]:
start = len(X_train[0])
h1 = 40
h2 = 15
#h3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=start, units=h1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=h2, activation="relu"))

# # Third hidden layer
# nn.add(tf.keras.layers.Dense(units=h3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_93 (Dense)             (None, 40)                1800      
_________________________________________________________________
dense_94 (Dense)             (None, 15)                615       
_________________________________________________________________
dense_95 (Dense)             (None, 1)                 16        
Total params: 2,431
Trainable params: 2,431
Non-trainable params: 0
_________________________________________________________________


In [119]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [120]:
fit_model = nn.fit(X_train,y_train,epochs=35)

Epoch 1/35
804/804 [==============================] - 1s 830us/step - loss: 21654.6953 - accuracy: 0.5019
Epoch 2/35
804/804 [==============================] - 1s 873us/step - loss: 14096.4346 - accuracy: 0.5040
Epoch 3/35
804/804 [==============================] - 1s 879us/step - loss: 30244.9434 - accuracy: 0.4907
Epoch 4/35
804/804 [==============================] - 1s 895us/step - loss: 13518.8975 - accuracy: 0.5074
Epoch 5/35
804/804 [==============================] - 1s 867us/step - loss: 11348.3535 - accuracy: 0.5190
Epoch 6/35
804/804 [==============================] - 1s 878us/step - loss: 27884.7832 - accuracy: 0.4941
Epoch 7/35
804/804 [==============================] - 1s 704us/step - loss: 9510.2793 - accuracy: 0.5077
Epoch 8/35
804/804 [==============================] - 1s 693us/step - loss: 6380.9062 - accuracy: 0.5264
Epoch 9/35
804/804 [==============================] - 1s 695us/step - loss: 7917.3726 - accuracy: 0.5152
Epoch 10/35
804/804 [============================

In [121]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6588 - accuracy: 0.6309
Loss: 0.6588335037231445, Accuracy: 0.6309037804603577


In [122]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

# Attempt #3 to Optimize

In [113]:
start = len(X_train[0])
h1 = 40
h2 = 15
h3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(input_dim=start, units=h1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=h2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=h3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_89 (Dense)             (None, 40)                1800      
_________________________________________________________________
dense_90 (Dense)             (None, 15)                615       
_________________________________________________________________
dense_91 (Dense)             (None, 10)                160       
_________________________________________________________________
dense_92 (Dense)             (None, 1)                 11        
Total params: 2,586
Trainable params: 2,586
Non-trainable params: 0
_________________________________________________________________


In [114]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [115]:
fit_model = nn.fit(X_train,y_train,epochs=29)

Epoch 1/29
804/804 [==============================] - 1s 884us/step - loss: 9310.4121 - accuracy: 0.4965
Epoch 2/29
804/804 [==============================] - 1s 939us/step - loss: 13512.6484 - accuracy: 0.4881
Epoch 3/29
804/804 [==============================] - 1s 909us/step - loss: 5616.6909 - accuracy: 0.4972
Epoch 4/29
804/804 [==============================] - 1s 948us/step - loss: 5623.7207 - accuracy: 0.5034
Epoch 5/29
804/804 [==============================] - 1s 946us/step - loss: 3214.0227 - accuracy: 0.4886
Epoch 6/29
804/804 [==============================] - 1s 961us/step - loss: 3951.2502 - accuracy: 0.4999
Epoch 7/29
804/804 [==============================] - 1s 975us/step - loss: 3029.8752 - accuracy: 0.5023
Epoch 8/29
804/804 [==============================] - 1s 971us/step - loss: 9510.3496 - accuracy: 0.4900
Epoch 9/29
804/804 [==============================] - 1s 956us/step - loss: 9613.2031 - accuracy: 0.4925
Epoch 10/29
804/804 [==============================] -

In [116]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 1.0156 - accuracy: 0.5248
Loss: 1.0155813694000244, Accuracy: 0.524781346321106
